In [1]:
import numpy as np
import scipy.optimize

In [2]:
# number of states and number of actions
n_states = 3
n_actions = 2

In [3]:
# initial state distribution
mu = np.array([[1, 0, 0]]).T
mu

array([[1],
       [0],
       [0]])

In [4]:
# Build the upper bound coefficients for the action A
# define the reward matrix for action A
R_A = np.zeros((n_states, 1), np.float)
R_A[0, 0] = 1
R_A[1, 0] = 0
R_A[2, 0] = 0
R_A

array([[1.],
       [0.],
       [0.]])

In [5]:
# Define the transition matrix for action A
P_A = np.zeros((n_states, n_states), np.float)
P_A[0, 1] = 1
P_A[1, 0] = 1
P_A[2, 1] = 1
P_A

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [6]:
gamma = 0.9

In [7]:
# Upper bound A matrix for action A
A_up_A = gamma * P_A - np.eye(3,3)
A_up_A

array([[-1. ,  0.9,  0. ],
       [ 0.9, -1. ,  0. ],
       [ 0. ,  0.9, -1. ]])

In [8]:
# The same for action B
# define the reward matrix for action B
R_B = np.zeros((n_states, 1), np.float)
R_B[0, 0] = 10
R_B[1, 0] = 1
R_B[2, 0] = 10
R_B
# Define the transition matrix for action A
P_B = np.zeros((n_states, n_states), np.float)
P_B[0, 2] = 1
P_B[1, 2] = 1
P_B[2, 2] = 1
P_B

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [9]:
# Upper bound A matrix for action B
A_up_B = gamma * P_B - np.eye(3,3)
A_up_B

array([[-1. ,  0. ,  0.9],
       [ 0. , -1. ,  0.9],
       [ 0. ,  0. , -0.1]])

In [10]:
# Upper bound matrix for all actions and all states
A_up = np.vstack((A_up_A, A_up_B))
# verify the shape: number of constraints are equal to |actions| * |states|
assert(A_up.shape[0] == n_states * n_actions)

In [11]:
# Reward vector is obtained by stacking the two vectors
R = np.vstack((R_A, R_B))

In [12]:
c = mu
b_up = -R
# Solve the linear program
res = scipy.optimize.linprog(c, A_up, b_up)

In [13]:
# Obtain the results: state values
V_ = res.x
V_
V = V_.reshape((-1, 1))
V
np.savetxt("solution/V.txt", V)

In [14]:
# transition matrix. On the rows we have states and actions, on the columns we have next states
P = np.vstack((P_A, P_B))
P

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [15]:
# Use the action value formula to calculate the action values for each state action pair.
Q_sa = R + gamma * P.dot(V)

In [16]:
# The first three rows are associated to action A, the last three are associated to action B
Q_sa

array([[ 88.32127683],
       [ 89.99999645],
       [ 87.32127683],
       [100.00000622],
       [ 91.00000622],
       [100.00000622]])

In [17]:
Q_sa_2 = np.stack((Q_sa[:3, 0], Q_sa[3:, 0]), axis=1)
Q_sa_2

array([[ 88.32127683, 100.00000622],
       [ 89.99999645,  91.00000622],
       [ 87.32127683, 100.00000622]])

In [18]:
best_actions = np.reshape(np.argmax(Q_sa_2, axis=1), (3, 1))

In [19]:
best_actions

array([[1],
       [1],
       [1]])